In [1]:
%%capture
pip install overpy

In [2]:
import pandas as pd

file_path1 = 'competitors_MI.csv'
competitors_MI = pd.read_csv(file_path1)

file_path2 = 'meijer_stores_MI.csv'
meijer_stores_MI = pd.read_csv(file_path2)

In [3]:
meijer_stores_MI.head(4)

,name,address,latitude,longitude,county
0,Meijer,"1251 M-32, Alpena, MI 49707, United States",45.059147,-83.474693,Alcona
1,Meijer,"100 Pigeon Rd, Bad Axe, MI 48413, United States",43.825699,-83.004745,Alcona
2,Meijer,"9515 Birch Run Rd, Birch Run, MI 48415, United...",43.252918,-83.765800,Alcona
3,Meijer,"3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...",43.482030,-83.982479,Alcona


In [4]:
competitors_MI.head(4)

,name,address,latitude,longitude,County
0,Kum & Go,"4800 44th St SW, Grandville, MI 49418, United ...",42.884595,-85.781344,Kent County
1,Kum & Go,"5437 Division Ave S, Wyoming, MI 49548, United...",42.865430,-85.664682,Kent County
2,Kum & Go,"4560 Chicago Dr SW, Grandville, MI 49418, Unit...",42.905083,-85.776313,Kent County
3,Kum & Go,"2134 Alpine Ave NW, Walker, MI 49544, United S...",43.002258,-85.688179,Kent County


In [5]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# Convert miles to meters for the radius
radius_miles = 3
radius = radius_miles * 1609.34

# Define categories and their tags
categories_tags = {
    
    "Grocery Stores": [
        ("amenity", "supermarket"),
        ("shop", "Bakery"),
        ("shop", "Convenience"),
        ("shop", "Frozen Food"),
        ("shop", "Department Store"),
        ("shop", "Supermarket"),
        ("shop", "Wholesale"),
    ],
    
    "Food Outlets": [
        ("amenity", "Fast Food"),
        ("amenity", "Food Court"),
        ("amenity", "restaurant"),
        ("amenity", "Ice Cream"),
        ("shop", "Cheese"),
        ("shop", "Chocolate"),
        ("shop", "Ice Cream"),
        ("shop", "Pasta"),
        ("shop", "Pastry"),
        ("shop", "Seafood"),
        ("shop", "Food"),
    ],
    
    "Tourist Destinations": [
        ("tourism", "attraction"),
        ("amenity", "Events Venue"),
        ("amenity", "Exhibition Centre"),
        ("boundary", "national_park"),
        ("boundary", "Museum"),
        ("building", "Beach Hut"),
        ("building", "Castle"),
    ],
    
    "Hotels": [
        ("building", "hotel"),
    ],
    
    "Educational Institutions": [
        ("building", "College"),
        ("building", "School"),
        ("amenity", "library"),
        ("building", "University"),
    ],
    
    "Hospitals": [
        ("amenity", "Clinic"),
        ("amenity", "Nursing Home"),
        ("amenity", "Pharmacy"),
        ("building", "hospital"),
    ],
    
    "Airports": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
    ],
    
    "Residential Housing Density": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "Hut"),
    ],
    
    "Commercial Housing Density": [
        ("building", "commercial"),
        ("building", "office"),
    ],
    
    "Fuel Stations & Car Wash": [
        ("amenity", "Car Wash"),
        ("amenity", "Charging Station"),
        ("amenity", "Fuel"),
    ],
    
    "Apparels": [
        ("shop", "Clothes"),
        ("shop", "Fabric"),
        ("shop", "Fashion"),
    ],
    
    "Height Restrictor": [
        ("barrier", "height_restrictor"),
    ],
    
    "Border Zone": [
        ("boundary", "border_zone"),
    ],
    
    "Public Transportation": [
        ("public_transport", "Public Transport Platform"),
        ("public_transport", "Public Transport Station"),
        ("public_transport", "Railway Station"),
        ("public_transport", "Railway Subway Entrance"),
        ("public_transport", "Railway Tram Stop"),
    ],
}


# Define a delay duration in seconds (e.g., 10 seconds)
delay_duration = 4

def fetch_feature_count(api, lat, lon, radius, category_tags):
    """
    Fetch count of features for the given category tags.

    :param api: Overpass API instance.
    :param lat: Latitude of the center.
    :param lon: Longitude of the center.
    :param radius: Radius for the search.
    :param category_tags: Dictionary of categories and their tags.
    :return: Dictionary with counts of features for each category.
    """
    category_counts = {}

    for category, tags in category_tags.items():
        print(f"Fetching data for category: {category}")
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count
        print(f"Completed fetching for category: {category}. Count: {category_count}")
        
        # Delay to manage server load
        print(f"Waiting for {delay_duration} seconds to manage server load.")
        time.sleep(delay_duration)

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        print(f"Processing location at latitude: {lat}, longitude: {lon}")
        feature_counts = fetch_feature_count(api, lat, lon, radius, categories_tags)
        feature_counts.update({
            "latitude": lat,
            "longitude": lon,
            "radius_miles": radius_miles
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    meijer_stores_MI_3Miles = pd.DataFrame(all_feature_counts)

    # Display the DataFrame
    print(meijer_stores_MI_3Miles)

    # Output the DataFrame to a CSV file
    meijer_stores_MI_3Miles.to_csv("meijer_nearby_places_final_3Miles.csv", index=False)
    print("Data collection complete. CSV file created.")

if __name__ == "__main__":
    # Input DataFrame with 'latitude' and 'longitude' columns
    meijer_nearby_places_final_3Miles = main(meijer_stores_MI)

Processing location at latitude: 45.0591468, longitude: -83.4746932
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 25
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetchi

Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 45.0611621, longitude: -83.4746063
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 25
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotel

Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 51
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 45.7514114, longitude: -87.0866699
Fetching 

Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 127
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 42
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height

Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 5
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 95
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Hous

...

Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 42.1914139, longitude: -83.1845448
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food O

Fetching data for category: Airports
Completed fetching for category: Airports. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 8
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category:

Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 3
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 3
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 4
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 5
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel S

Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 19
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Re

Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 42.2314156, longitude: -83.679345
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 45
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 8
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Com

In [6]:
file_path = 'meijer_nearby_places_final_3Miles.csv'
nearby_places_final_3Miles = pd.read_csv(file_path)
nearby_places_final_3Miles.head()

,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,latitude,longitude,radius_miles
0,0,25,2,1,1,1,1,88,28,0,0,0,0,0,45.059147,-83.474693,3
1,0,6,0,0,1,1,4,17,1,0,0,0,0,0,43.825699,-83.004745,3
2,0,8,0,1,0,0,0,0,2,0,0,0,0,0,43.252918,-83.765800,3
3,0,33,0,1,2,2,0,66,28,0,0,0,0,0,43.482030,-83.982479,3
4,0,25,2,1,1,1,1,88,28,0,0,0,0,0,45.061162,-83.474606,3


In [7]:
final_nearby_places_meijer_3Miles = pd.merge(meijer_stores_MI, nearby_places_final_3Miles, on=['latitude','longitude'], how='inner')
final_nearby_places_meijer_3Miles

,name,address,latitude,longitude,county,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,radius_miles
0,Meijer,"1251 M-32, Alpena, MI 49707, United States",45.059147,-83.474693,Alcona,0,25,2,1,1,1,1,88,28,0,0,0,0,0,3
1,Meijer,"100 Pigeon Rd, Bad Axe, MI 48413, United States",43.825699,-83.004745,Alcona,0,6,0,0,1,1,4,17,1,0,0,0,0,0,3
2,Meijer,"9515 Birch Run Rd, Birch Run, MI 48415, United...",43.252918,-83.765800,Alcona,0,8,0,1,0,0,0,0,2,0,0,0,0,0,3
3,Meijer,"3360 Tittabawassee Rd, Saginaw, MI 48604, Unit...",43.482030,-83.982479,Alcona,0,33,0,1,2,2,0,66,28,0,0,0,0,0,3
4,Meijer Express Gas Station,"1241 M-32, Alpena, MI 49707, United States",45.061162,-83.474606,Alcona,0,25,2,1,1,1,1,88,28,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Meijer Express Gas Station,"521 N Pine Rd, Bay City, MI 48708, United States",43.580492,-83.837882,Tuscola,0,18,0,0,1,0,2,4,11,0,0,0,0,0,3
204,Meijer Express Gas Station,"5671 Jackson Rd, Ann Arbor, MI 48103, United S...",42.287728,-83.842059,Washtenaw,0,10,0,2,0,0,0,217,16,0,0,0,0,0,3
205,Meijer Express Gas Station,"3995 Carpenter Rd, Ypsilanti, MI 48197, United...",42.231416,-83.679345,Washtenaw,0,45,1,8,4,10,1,776,106,0,0,0,0,0,3
206,Meijer,"1475 E Jefferson Ave, Detroit, MI 48207, Unite...",42.334396,-83.032600,Wayne,0,253,16,11,18,17,2,834,418,0,0,0,0,0,3


In [8]:
final_nearby_places_meijer_3Miles.to_csv("final_nearby_places_meijer_3Miles.csv", index=False)

# Competitor Stores

In [9]:
import overpy
import pandas as pd
import time

# Initialize Overpass API
api = overpy.Overpass()

# Convert miles to meters for the radius
radius_miles = 3
radius = radius_miles * 1609.34

# Define categories and their tags
categories_tags = {
    
    "Grocery Stores": [
        ("amenity", "supermarket"),
        ("shop", "Bakery"),
        ("shop", "Convenience"),
        ("shop", "Frozen Food"),
        ("shop", "Department Store"),
        ("shop", "Supermarket"),
        ("shop", "Wholesale"),
    ],
    
    "Food Outlets": [
        ("amenity", "Fast Food"),
        ("amenity", "Food Court"),
        ("amenity", "restaurant"),
        ("amenity", "Ice Cream"),
        ("shop", "Cheese"),
        ("shop", "Chocolate"),
        ("shop", "Ice Cream"),
        ("shop", "Pasta"),
        ("shop", "Pastry"),
        ("shop", "Seafood"),
        ("shop", "Food"),
    ],
    
    "Tourist Destinations": [
        ("tourism", "attraction"),
        ("amenity", "Events Venue"),
        ("amenity", "Exhibition Centre"),
        ("boundary", "national_park"),
        ("boundary", "Museum"),
        ("building", "Beach Hut"),
        ("building", "Castle"),
    ],
    
    "Hotels": [
        ("building", "hotel"),
    ],
    
    "Educational Institutions": [
        ("building", "College"),
        ("building", "School"),
        ("amenity", "library"),
        ("building", "University"),
    ],
    
    "Hospitals": [
        ("amenity", "Clinic"),
        ("amenity", "Nursing Home"),
        ("amenity", "Pharmacy"),
        ("building", "hospital"),
    ],
    
    "Airports": [
        ("aeroway", "helipad"),
        ("aeroway", "heliport"),
        ("aeroway", "runway"),
        ("aeroway", "terminal"),
    ],
    
    "Residential Housing Density": [
        ("building", "bungalow"),
        ("building", "apartments"),
        ("building", "residential"),
        ("building", "Hut"),
    ],
    
    "Commercial Housing Density": [
        ("building", "commercial"),
        ("building", "office"),
    ],
    
    "Fuel Stations & Car Wash": [
        ("amenity", "Car Wash"),
        ("amenity", "Charging Station"),
        ("amenity", "Fuel"),
    ],
    
    "Apparels": [
        ("shop", "Clothes"),
        ("shop", "Fabric"),
        ("shop", "Fashion"),
    ],
    
    "Height Restrictor": [
        ("barrier", "height_restrictor"),
    ],
    
    "Border Zone": [
        ("boundary", "border_zone"),
    ],
    
    "Public Transportation": [
        ("public_transport", "Public Transport Platform"),
        ("public_transport", "Public Transport Station"),
        ("public_transport", "Railway Station"),
        ("public_transport", "Railway Subway Entrance"),
        ("public_transport", "Railway Tram Stop"),
    ],
}


# Define a delay duration in seconds (e.g., 10 seconds)
delay_duration = 4

def fetch_feature_count(api, lat, lon, radius, category_tags):
    """
    Fetch count of features for the given category tags.

    :param api: Overpass API instance.
    :param lat: Latitude of the center.
    :param lon: Longitude of the center.
    :param radius: Radius for the search.
    :param category_tags: Dictionary of categories and their tags.
    :return: Dictionary with counts of features for each category.
    """
    category_counts = {}

    for category, tags in category_tags.items():
        print(f"Fetching data for category: {category}")
        category_count = 0
        for tag_pair in tags:
            category_name, tag_name = tag_pair
            query = f"""
                (node["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 way["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                 rel["{category_name}"="{tag_name}"](around:{radius},{lat},{lon});
                );out;
            """
            result = api.query(query)
            category_count += len(result.nodes) + len(result.ways) + len(result.relations)
        
        category_counts[category] = category_count
        print(f"Completed fetching for category: {category}. Count: {category_count}")
        
        # Delay to manage server load
        print(f"Waiting for {delay_duration} seconds to manage server load.")
        time.sleep(delay_duration)

    return category_counts

def main(input_df):
    all_feature_counts = []

    for index, row in input_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        print(f"Processing location at latitude: {lat}, longitude: {lon}")
        feature_counts = fetch_feature_count(api, lat, lon, radius, categories_tags)
        feature_counts.update({
            "latitude": lat,
            "longitude": lon,
            "radius_miles": radius_miles
        })
        all_feature_counts.append(feature_counts)

    # Convert the list of dictionaries to a DataFrame
    competitors_MI_3Miles = pd.DataFrame(all_feature_counts)

    # Display the DataFrame
    print(competitors_MI_3Miles)

    # Output the DataFrame to a CSV file
    competitors_MI_3Miles.to_csv("competitor_nearby_places_final_3Miles.csv", index=False)
    print("Data collection complete. CSV file created.")

if __name__ == "__main__":
    # Input DataFrame with 'latitude' and 'longitude' columns
    competitor_nearby_places_final_3Miles = main(competitors_MI)

Processing location at latitude: 42.8845946, longitude: -85.7813442
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 34
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetchi

Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 42.8688229, longitude: -85.6257608
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 17
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotel

Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 57
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 42.7660971, longitude: -85.3165607
Fetching 

Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Re

Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 15
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 46
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Hou

...

Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 42.2607092, longitude: -83.6026435
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 36
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Co

Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for category: Border Zone. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Public Transportation
Completed fetching for category: Public Transportation. Count: 0
Waiting for 4 seconds to manage server load.
Processing location at latitude: 45.6281129, longitude: -84.4781093
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food O

Fetching data for category: Airports
Completed fetching for category: Airports. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 362
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 5
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fuel Stations & Car Wash. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Apparels
Completed fetching for category: Apparels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Height Restrictor
Completed fetching for category: Height Restrictor. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Border Zone
Completed fetching for categor

Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 5
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 4
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetching for category: Airports. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Residential Housing Density
Completed fetching for category: Residential Housing Density. Count: 293
Waiting for 4 seconds to manage server load.
Fetching data for category: Commercial Housing Density
Completed fetching for category: Commercial Housing Density. Count: 39
Waiting for 4 seconds to manage server load.
Fetching data for category: Fuel Stations & Car Wash
Completed fetching for category: Fue

Processing location at latitude: 42.2980167, longitude: -83.2853154
Fetching data for category: Grocery Stores
Completed fetching for category: Grocery Stores. Count: 0
Waiting for 4 seconds to manage server load.
Fetching data for category: Food Outlets
Completed fetching for category: Food Outlets. Count: 63
Waiting for 4 seconds to manage server load.
Fetching data for category: Tourist Destinations
Completed fetching for category: Tourist Destinations. Count: 2
Waiting for 4 seconds to manage server load.
Fetching data for category: Hotels
Completed fetching for category: Hotels. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Educational Institutions
Completed fetching for category: Educational Institutions. Count: 5
Waiting for 4 seconds to manage server load.
Fetching data for category: Hospitals
Completed fetching for category: Hospitals. Count: 1
Waiting for 4 seconds to manage server load.
Fetching data for category: Airports
Completed fetchi

In [10]:
file_path = 'competitor_nearby_places_final_3Miles.csv'
competitor_nearby_places_final_3Miles = pd.read_csv(file_path)
competitor_nearby_places_final_3Miles.head()

,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,latitude,longitude,radius_miles
0,0,34,0,1,2,0,1,159,86,0,0,0,0,0,42.884595,-85.781344,3
1,0,25,0,1,1,4,1,59,9,0,0,0,0,0,42.865430,-85.664682,3
2,0,36,1,1,2,0,2,119,84,0,0,0,0,0,42.905083,-85.776313,3
3,0,88,0,2,5,3,4,72,53,0,0,0,0,0,43.002258,-85.688179,3
4,0,17,0,0,2,0,0,189,10,0,0,0,0,0,42.868823,-85.625761,3


In [11]:
final_competitor_nearby_places_3Miles = pd.merge(competitors_MI, competitor_nearby_places_final_3Miles, on=['latitude','longitude'], how='inner')
final_competitor_nearby_places_3Miles

,name,address,latitude,longitude,County,Grocery Stores,Food Outlets,Tourist Destinations,Hotels,Educational Institutions,Hospitals,Airports,Residential Housing Density,Commercial Housing Density,Fuel Stations & Car Wash,Apparels,Height Restrictor,Border Zone,Public Transportation,radius_miles
0,Kum & Go,"4800 44th St SW, Grandville, MI 49418, United ...",42.884595,-85.781344,Kent County,0,34,0,1,2,0,1,159,86,0,0,0,0,0,3
1,Kum & Go,"5437 Division Ave S, Wyoming, MI 49548, United...",42.865430,-85.664682,Kent County,0,25,0,1,1,4,1,59,9,0,0,0,0,0,3
2,Kum & Go,"4560 Chicago Dr SW, Grandville, MI 49418, Unit...",42.905083,-85.776313,Kent County,0,36,1,1,2,0,2,119,84,0,0,0,0,0,3
3,Kum & Go,"2134 Alpine Ave NW, Walker, MI 49544, United S...",43.002258,-85.688179,Kent County,0,88,0,2,5,3,4,72,53,0,0,0,0,0,3
4,Kum & Go,"5215 Kalamazoo Ave SE, Kentwood, MI 49508, Uni...",42.868823,-85.625761,Kent County,0,17,0,0,2,0,0,189,10,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,Circle K,"27248 Van Dyke Ave, Warren, MI 48093, United S...",42.493987,-83.027142,Macomb County,0,38,0,0,5,4,2,293,39,0,0,1,0,0,3
281,Circle K,"1420 E Stadium Blvd, Ann Arbor, MI 48104, Unit...",42.260417,-83.730209,Washtenaw County,0,198,9,17,14,8,7,1383,220,0,0,0,0,0,3
282,Circle K,"1879 N Lapeer Rd, Lapeer, MI 48446, United States",43.083855,-83.308497,Lapeer County,0,10,0,0,1,1,3,0,1,0,0,0,0,0,3
283,Circle K,"12800 Hamilton Ave, Highland Park, MI 48203, U...",42.396613,-83.101893,Wayne County,0,40,2,2,10,11,1,614,132,0,0,0,0,0,3


In [12]:
 final_competitor_nearby_places_3Miles.to_csv("final_competitor_nearby_places_3Miles.csv", index=False)

# Final Nearby Places DF

In [13]:
import pandas as pd

# Standardize column names to lowercase for both DataFrames
final_competitor_nearby_places_3Miles.columns = final_competitor_nearby_places_3Miles.columns.str.lower()
final_nearby_places_meijer_3Miles.columns = final_nearby_places_meijer_3Miles.columns.str.lower()

# Now concatenate the DataFrames
final_nearby_places_df_3Miles = pd.concat([final_competitor_nearby_places_3Miles, final_nearby_places_meijer_3Miles], ignore_index=True)

# List of all columns to edit with the '_1mile' suffix
columns_to_edit = [
    'grocery stores', 'food outlets', 'tourist destinations', 'hotels', 
    'educational institutions', 'hospitals', 'airports', 
    'residential housing density', 'commercial housing density', 
    'fuel stations & car wash', 'apparels', 'height restrictor', 
    'border bone', 'public transportation'
]

# Creating the dictionary for renaming
rename_dict = {col: col + ' 3 mile' for col in columns_to_edit}

# Renaming the columns
final_nearby_places_df_3Miles.rename(columns=rename_dict, inplace=True)

# Removing the 'radius_miles' column
final_nearby_places_df_3Miles.drop(columns=['radius_miles'], inplace=True)

final_nearby_places_df_3Miles.to_csv("final_nearby_places_df_3Miles.csv", index=False)

final_nearby_places_df_3Miles

,name,address,latitude,longitude,county,grocery stores 3 mile,food outlets 3 mile,tourist destinations 3 mile,hotels 3 mile,educational institutions 3 mile,hospitals 3 mile,airports 3 mile,residential housing density 3 mile,commercial housing density 3 mile,fuel stations & car wash 3 mile,apparels 3 mile,height restrictor 3 mile,border zone,public transportation 3 mile
0,Kum & Go,"4800 44th St SW, Grandville, MI 49418, United ...",42.884595,-85.781344,Kent County,0,34,0,1,2,0,1,159,86,0,0,0,0,0
1,Kum & Go,"5437 Division Ave S, Wyoming, MI 49548, United...",42.865430,-85.664682,Kent County,0,25,0,1,1,4,1,59,9,0,0,0,0,0
2,Kum & Go,"4560 Chicago Dr SW, Grandville, MI 49418, Unit...",42.905083,-85.776313,Kent County,0,36,1,1,2,0,2,119,84,0,0,0,0,0
3,Kum & Go,"2134 Alpine Ave NW, Walker, MI 49544, United S...",43.002258,-85.688179,Kent County,0,88,0,2,5,3,4,72,53,0,0,0,0,0
4,Kum & Go,"5215 Kalamazoo Ave SE, Kentwood, MI 49508, Uni...",42.868823,-85.625761,Kent County,0,17,0,0,2,0,0,189,10,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,Meijer Express Gas Station,"521 N Pine Rd, Bay City, MI 48708, United States",43.580492,-83.837882,Tuscola,0,18,0,0,1,0,2,4,11,0,0,0,0,0
489,Meijer Express Gas Station,"5671 Jackson Rd, Ann Arbor, MI 48103, United S...",42.287728,-83.842059,Washtenaw,0,10,0,2,0,0,0,217,16,0,0,0,0,0
490,Meijer Express Gas Station,"3995 Carpenter Rd, Ypsilanti, MI 48197, United...",42.231416,-83.679345,Washtenaw,0,45,1,8,4,10,1,776,106,0,0,0,0,0
491,Meijer,"1475 E Jefferson Ave, Detroit, MI 48207, Unite...",42.334396,-83.032600,Wayne,0,253,16,11,18,17,2,834,418,0,0,0,0,0
